In [246]:
"""
Новая версия.
"""

'\nНовая версия.\n'

In [247]:
import tkinter
import sys
import pandas as pd
import openpyxl
import numpy as np
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
import os
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
import time
import datetime
# pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.options.mode.chained_assignment = None  # default='warn'


In [248]:
name_file_abiturs = 'data/abitur.xlsx'
name_file_person = 'data/person.xlsx'
path_to_end_folder_report = 'data'

In [249]:
#создаем датафрейм со специальностями
lst_code = ['23.01.09','43.01.06','23.02.06','43.02.06','15.01.05','15.01.35','15.01.33','23.01.10'
    ,'08.01.31','23.01.17','08.02.09','23.02.07','13.02.07','35.01.27']

lst_name_spec = ['Машинист локомотива','Проводник на железнодорожном транспорте','Техническая эксплуатация подвижного состава железных дорог',
                 'Сервис на транспорте (по видам транспорта)','Сварщик (ручной и частично механизированной сварки (наплавки)',
                 'Мастер слесарных работ','Токарь на станках с числовым программным управлением','Слесарь по обслуживанию и ремонту подвижного состава'
    ,'Электромонтажник электрических сетей и электрооборудования','Мастер по ремонту и обслуживанию автомобилей',
                 'Монтаж, наладка и эксплуатация электрооборудования промышленных и гражданских зданий',
                 'Техническое обслуживание и ремонт двигателей, систем и агрегатов автомобилей','Электроснабжение (по отраслям)',
                 'Мастер сельскохозяйственного производства']

lst_plan = [100,25,25,25,50,25,25,50
    ,25,25,25,25,25,25]
base_df = pd.DataFrame(columns=['Код','Наименование'])
base_df['Код'] = lst_code
base_df['Наименование'] = lst_name_spec
base_df['Направление подготовки'] = base_df['Код'] + ' ' + base_df['Наименование']
base_df['База'] = '9 кл.'
base_df['Количество мест'] = lst_plan

In [250]:
df_abitur = pd.read_excel(name_file_abiturs, skiprows=3, usecols=['Абитуриент', 'Доп. статус','№ заявления'])
df_person = pd.read_excel(name_file_person, sheet_name='Абитуриенты', skiprows=8,
                          usecols=['ФИО', 'Нуждается в общежитии', 'Формирующее подр.',
                                   'Направление подготовки', 'Сдан оригинал','Состояние выбран. конкурса'])



In [251]:
df_person = df_person[~df_person['Направление подготовки'].isnull()]  # убираем тех у кого нет заявлений
df_abitur = df_abitur[~df_abitur['№ заявления'].isnull()]



In [252]:
df_dupl = df_person.drop_duplicates(subset=['ФИО'])  # создаем датафрейм без дубликатов

In [253]:



dupl_cross_df = df_dupl.merge(df_abitur, how='inner', left_on='ФИО', right_on='Абитуриент')


In [254]:
# Преобразовываем да-нет в 1 или 0 для подсчетов
dupl_cross_df['Нуждается в общежитии'] = dupl_cross_df['Нуждается в общежитии'].apply(
    lambda x: 0 if x == 'нет' else 1)
dupl_cross_df['Сдан оригинал'] = dupl_cross_df['Сдан оригинал'].apply(lambda x: 0 if x == 'нет' else 1)
# заменяем нан на пустые строки чтобы произвести поиск слова сирота;
dupl_cross_df['Доп. статус'].fillna('', inplace=True)
dupl_cross_df['Сироты'] = dupl_cross_df['Доп. статус'].apply(lambda x: 1 if 'Сирота;' in x else 0)
dupl_cross_df['СВО'] = dupl_cross_df['Доп. статус'].apply(
    lambda x: 1 if 'Дети военнослужащих, участвующих в спецоперации' in x else 0)
dupl_cross_df['Целевой договор'] = dupl_cross_df['Доп. статус'].apply(lambda x: 1 if 'Целевой договор' in x else 0)

dupl_cross_df['for_counting'] = 1

dupl_cross_df.drop(columns=['Доп. статус'], inplace=True)

In [255]:
ts3=dupl_cross_df[dupl_cross_df['Направление подготовки'] == '35.01.27 Мастер сельскохозяйственного производства']

In [256]:
ts3.shape

(18, 12)

In [257]:
dupl_svod_df = pd.DataFrame.pivot_table(dupl_cross_df,
                                        index=['Формирующее подр.', 'Направление подготовки'],
                                        values=['Сдан оригинал', 'Сироты', 'СВО','Целевой договор',
                                                'Нуждается в общежитии'],
                                        aggfunc='sum')

dupl_svod_df.columns = ['Нуждается в общежитии чел.', 'Дети СВО', 'Сдано оригиналов', 'Сирот чел.','Целевой договор']

In [258]:
dupl_svod_df

Нуждается в общежитии чел.  \
Формирующее подр.                     Направление подготовки                                                           
Отделение железнодорожного транспорта 23.01.09 Машинист локомотива                                                46   
                                      23.02.06 Техническая эксплуатация подвижного со...                          11   
                                      43.01.06 Проводник на железнодорожном транспорте                            36   
                                      43.02.06 Сервис на транспорте (по видам транспо...                          26   
Отделение металлообработки            15.01.05 Сварщик (ручной и частично механизиров...                          27   
                                      15.01.33 Токарь на станках с числовым программн...                          17   
                                      15.01.35 Мастер слесарных работ                                              8   
                                      23.01.10 Слесарь по обслуживанию и ремонту подв...                          28   
Отделение энергоснабжения             08.01.31 Электромонтажник электрических сетей и...                           6   
                                      08.02.09 Монтаж, наладка и эксплуатация электро...                           8   
                                      13.02.07 Электроснабжение (по отраслям)                                      6   
                                      23.01.17 Мастер по ремонту и обслуживанию автом...                          20   
                                      23.02.07 Техническое обслуживание и ремонт двиг...                          11   
                                      38.02.02 Страховое дело (по отраслям)                                        2   
Хоринский филиал                      35.01.27 Мастер сельскохозяйственного производства                           6   

                                                                                          Дети СВО  \
Формирующее подр.                     Направление подготовки                                         
Отделение железнодорожного транспорта 23.01.09 Машинист локомотива                               2   
                                      23.02.06 Техническая эксплуатация подвижного со...         1   
                                      43.01.06 Проводник на железнодорожном транспорте           1   
                                      43.02.06 Сервис на транспорте (по видам транспо...         1   
Отделение металлообработки            15.01.05 Сварщик (ручной и частично механизиров...         1   
                                      15.01.33 Токарь на станках с числовым программн...         0   
                                      15.01.35 Мастер слесарных работ                            0   
                                      23.01.10 Слесарь по обслуживанию и ремонту подв...         1   
Отделение энергоснабжения             08.01.31 Электромонтажник электрических сетей и...         0   
                                      08.02.09 Монтаж, наладка и эксплуатация электро...         0   
                                      13.02.07 Электроснабжение (по отраслям)                    1   
                                      23.01.17 Мастер по ремонту и обслуживанию автом...         0   
                                      23.02.07 Техническое обслуживание и ремонт двиг...         0   
                                      38.02.02 Страховое дело (по отраслям)                      0   
Хоринский филиал                      35.01.27 Мастер сельскохозяйственного производства         0   

                                                                                          Сдано оригиналов  \
Формирующее подр.                     Направление подготовки                                                 
Отделение железнодорожного транспорта 23.01.09 Машинист локомотива                            

In [259]:
# Меняем местами столбцы
single_out_df = dupl_svod_df.reindex(
    columns=['Сдано оригиналов',
             'Нуждается в общежитии чел.',
             'Сирот чел.', 'Дети СВО','Целевой договор'])

In [260]:
single_out_df

Сдано оригиналов  \
Формирующее подр.                     Направление подготовки                                                 
Отделение железнодорожного транспорта 23.01.09 Машинист локомотива                                     125   
                                      23.02.06 Техническая эксплуатация подвижного со...                12   
                                      43.01.06 Проводник на железнодорожном транспорте                  56   
                                      43.02.06 Сервис на транспорте (по видам транспо...                46   
Отделение металлообработки            15.01.05 Сварщик (ручной и частично механизиров...                61   
                                      15.01.33 Токарь на станках с числовым программн...                31   
                                      15.01.35 Мастер слесарных работ                                   16   
                                      23.01.10 Слесарь по обслуживанию и ремонту подв...                37   
Отделение энергоснабжения             08.01.31 Электромонтажник электрических сетей и...                14   
                                      08.02.09 Монтаж, наладка и эксплуатация электро...                22   
                                      13.02.07 Электроснабжение (по отраслям)                           14   
                                      23.01.17 Мастер по ремонту и обслуживанию автом...                38   
                                      23.02.07 Техническое обслуживание и ремонт двиг...                15   
                                      38.02.02 Страховое дело (по отраслям)                              4   
Хоринский филиал                      35.01.27 Мастер сельскохозяйственного производства                18   

                                                                                          Нуждается в общежитии чел.  \
Формирующее подр.                     Направление подготовки                                                           
Отделение железнодорожного транспорта 23.01.09 Машинист локомотива                                                46   
                                      23.02.06 Техническая эксплуатация подвижного со...                          11   
                                      43.01.06 Проводник на железнодорожном транспорте                            36   
                                      43.02.06 Сервис на транспорте (по видам транспо...                          26   
Отделение металлообработки            15.01.05 Сварщик (ручной и частично механизиров...                          27   
                                      15.01.33 Токарь на станках с числовым программн...                          17   
                                      15.01.35 Мастер слесарных работ                                              8   
                                      23.01.10 Слесарь по обслуживанию и ремонту подв...                          28   
Отделение энергоснабжения             08.01.31 Электромонтажник электрических сетей и...                           6   
                                      08.02.09 Монтаж, наладка и эксплуатация электро...                           8   
                                      13.02.07 Электроснабжение (по отраслям)                                      6   
                                      23.01.17 Мастер по ремонту и обслуживанию автом...                          20   
                                      23.02.07 Техническое обслуживание и ремонт двиг...                          11   
                                      38.02.02 Страховое дело (по отраслям)                                        2   
Хоринский филиал                      35.01.27 Мастер сельскохозяйственного производства                           6   

                                                                                          Сирот чел.  \
Формирующее подр.                     Направление подготовки              

In [261]:
# Соединяем оба датафрейма

cross_df = df_person.merge(df_abitur, how='inner', left_on='ФИО', right_on='Абитуриент')

In [262]:
cross_df[cross_df['Направление подготовки'] == '08.01.31 Электромонтажник электрических сетей и электрооборудования']

,ФИО,Нуждается в общежитии,Формирующее подр.,Направление подготовки,Состояние выбран. конкурса,Сдан оригинал,№ заявления,Абитуриент,Доп. статус
39,Бабошина Вера Вячеславовна,нет,Отделение энергоснабжения,08.01.31 Электромонтажник электрических сетей ...,Сданы ВИ,нет,534,Бабошина Вера Вячеславовна,NaN
47,Бадмаев Кирилл Евгеньевич,нет,Отделение энергоснабжения,08.01.31 Электромонтажник электрических сетей ...,Сданы ВИ,да,211,Бадмаев Кирилл Евгеньевич,NaN
61,Банаев Альберт Баторович,нет,Отделение энергоснабжения,08.01.31 Электромонтажник электрических сетей ...,Сданы ВИ,нет,368,Банаев Альберт Баторович,NaN
82,Березовский Дмитрий Сергеевич,да,Отделение энергоснабжения,08.01.31 Электромонтажник электрических сетей ...,Сданы ВИ,да,195,Березовский Дмитрий Сергеевич,NaN
88,Богидаева Елена Григорьевна,да,Отделение энергоснабжения,08.01.31 Электромонтажник электрических сетей ...,Сданы ВИ,да,231,Богидаева Елена Григорьевна,NaN
108,Будаев Даши Вячеславович,да,Отделение энергоснабжения,08.01.31 Электромонтажник электрических сетей ...,Сданы ВИ,да,338,Будаев Даши Вячеславович,NaN
115,Бурлаков Данила Витальевич,да,Отделение энергоснабжения,08.01.31 Электромонтажник электрических сетей ...,Сданы ВИ,да,120,Бурлаков Данила Витальевич,NaN
124,Васильев Антон Андреевич,нет,Отделение энергоснабжения,08.01.31 Электромонтажник электрических сетей ...,Сданы ВИ,да,80,Васильев Антон Андреевич,NaN
158,Ганжуров Бэлигто Буладович,да,Отделение энергоснабжения,08.01.31 Электромонтажник электрических сетей ...,Сданы ВИ,да,87,Ганжуров Бэлигто Буладович,NaN
167,Гармаев Тимур Сергеевич,нет,Отделение энергоснабжения,08.01.31 Электромонтажник электрических сетей ...,Сданы ВИ,да,481,Гармаев Тимур Сергеевич,NaN


In [263]:
ts4=cross_df[cross_df['Направление подготовки'] == '08.01.31 Электромонтажник электрических сетей и электрооборудования']

In [264]:
ts4.shape

(34, 9)

In [265]:
# Преобразовываем да-нет в 1 или 0 для подсчетов
cross_df['Нуждается в общежитии'] = cross_df['Нуждается в общежитии'].apply(lambda x: 0 if x == 'нет' else 1)
cross_df['Сдан оригинал'] = cross_df['Сдан оригинал'].apply(lambda x: 0 if x == 'нет' else 1)

In [266]:
# заменяем нан на пустые строки чтобы произвести поиск слова сирота;
cross_df['Доп. статус'].fillna('', inplace=True)
cross_df['Сироты'] = cross_df['Доп. статус'].apply(lambda x: 1 if 'Сирота;' in x else 0)
cross_df['СВО'] = cross_df['Доп. статус'].apply(
    lambda x: 1 if 'Дети военнослужащих, участвующих в спецоперации' in x else 0)

In [267]:
cross_df['for_counting'] = 1

cross_df.drop(columns=['Доп. статус'], inplace=True)

In [268]:
cross_df

,ФИО,Нуждается в общежитии,Формирующее подр.,Направление подготовки,Состояние выбран. конкурса,Сдан оригинал,№ заявления,Абитуриент,Сироты,СВО,for_counting
0,Абросова Анастасия Витальевна,0,Отделение железнодорожного транспорта,43.01.06 Проводник на железнодорожном транспорте,Сданы ВИ,1,60,Абросова Анастасия Витальевна,0,0,1
1,Авдеев Сергей Александрович,0,Отделение энергоснабжения,23.01.17 Мастер по ремонту и обслуживанию авто...,Сданы ВИ,1,447,Авдеев Сергей Александрович,0,0,1
2,Авдеев Сергей Александрович,0,Отделение металлообработки,23.01.10 Слесарь по обслуживанию и ремонту под...,Сданы ВИ,1,447,Авдеев Сергей Александрович,0,0,1
3,Авсеевич Радмила Станиславовна,1,Отделение металлообработки,15.01.33 Токарь на станках с числовым программ...,Сданы ВИ,1,637,Авсеевич Радмила Станиславовна,0,0,1
4,Агафонов Семен Владимирович,0,Отделение железнодорожного транспорта,23.01.09 Машинист локомотива,Сданы ВИ,1,418,Агафонов Семен Владимирович,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
826,Яшин Никита Денисович,0,Отделение металлообработки,15.01.05 Сварщик (ручной и частично механизиро...,Забрал документы,1,105\n198,Яшин Никита Денисович,0,0,1
827,Яшин Никита Денисович,0,Отделение металлообработки,23.01.10 Слесарь по обслуживанию и ремонту под...,Забрал документы,1,105\n198,Яшин Никита Денисович,0,0,1
828,Яшин Никита Денисович,0,Отделение металлообработки,15.01.35 Мастер слесарных работ,Забрал документы,1,105\n198,Яшин Никита Денисович,0,0,1
829,Яшин Никита Денисович,0,Отделение металлообработки,15.01.05 Сварщик (ручной и частично механизиро...,Сданы ВИ,1,105\n198,Яшин Никита Денисович,0,0,1


In [269]:
# Создаем сокращенный датафрейм чтобы добавить его в базовый
small_df = cross_df[['Направление подготовки','Состояние выбран. конкурса','for_counting']]

In [270]:
# объединяем датафреймы
union_df = base_df.merge(small_df,how='outer',left_on='Направление подготовки',right_on='Направление подготовки')
union_df.fillna(0,inplace=True)

In [271]:
# забранные заявления
return_z = union_df[union_df['Состояние выбран. конкурса'] =='Забрал документы' ]

In [272]:
union_df.groupby(['Направление подготовки']).agg({'for_counting':sum})

,for_counting
Направление подготовки,
08.01.31 Электромонтажник электрических сетей и электрооборудования,34
"08.02.09 Монтаж, наладка и эксплуатация электрооборудования промышленных и гражданских зданий",33
13.02.07 Электроснабжение (по отраслям),39
15.01.05 Сварщик (ручной и частично механизированной сварки (наплавки),95
15.01.33 Токарь на станках с числовым программным управлением,58
15.01.35 Мастер слесарных работ,32
23.01.09 Машинист локомотива,169
23.01.10 Слесарь по обслуживанию и ремонту подвижного состава,64
23.01.17 Мастер по ремонту и обслуживанию автомобилей,74


In [273]:
base_df

,Код,Наименование,Направление подготовки,База,Количество мест
0,23.01.09,Машинист локомотива,23.01.09 Машинист локомотива,9 кл.,100
1,43.01.06,Проводник на железнодорожном транспорте,43.01.06 Проводник на железнодорожном транспорте,9 кл.,25
2,23.02.06,Техническая эксплуатация подвижного состава же...,23.02.06 Техническая эксплуатация подвижного с...,9 кл.,25
3,43.02.06,Сервис на транспорте (по видам транспорта),43.02.06 Сервис на транспорте (по видам трансп...,9 кл.,25
4,15.01.05,Сварщик (ручной и частично механизированной св...,15.01.05 Сварщик (ручной и частично механизиро...,9 кл.,50
5,15.01.35,Мастер слесарных работ,15.01.35 Мастер слесарных работ,9 кл.,25
6,15.01.33,Токарь на станках с числовым программным управ...,15.01.33 Токарь на станках с числовым программ...,9 кл.,25
7,23.01.10,Слесарь по обслуживанию и ремонту подвижного с...,23.01.10 Слесарь по обслуживанию и ремонту под...,9 кл.,50
8,08.01.31,Электромонтажник электрических сетей и электро...,08.01.31 Электромонтажник электрических сетей ...,9 кл.,25
9,23.01.17,Мастер по ремонту и обслуживанию автомобилей,23.01.17 Мастер по ремонту и обслуживанию авто...,9 кл.,25


In [274]:
base_df_groupby = union_df.groupby(['Наименование']).agg({'for_counting':sum})
base_df_groupby['for_counting'] = base_df_groupby['for_counting'].apply(int)
base_df_groupby = base_df_groupby.reset_index()
base_df_groupby.rename(columns={'for_counting':'Подано заявлений'},inplace=True)

base_df = base_df.merge(base_df_groupby,how='inner',left_on='Наименование',right_on='Наименование')
base_df.sort_values(by='Наименование',inplace=True)
base_df.rename(columns={'Наименование':'Наименование образовательной программы'})
base_df.drop(columns='Направление подготовки',inplace=True)
